In [5]:
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import os
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader
import pickle
import numpy as np
import torch
from sklearn import metrics

In [2]:
import torch.optim as optim

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
class CustomEEGDataset(Dataset):
    def __init__(self, annotations_file, eeg_file, transform=None, target_transform=None):
        self.eeg_labels = torch.from_numpy(np.load(annotations_file).reshape(-1,1))
        self.eeg_data = torch.from_numpy(np.load(eeg_file))
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.eeg_labels)

    def __getitem__(self, idx):
        label = self.eeg_labels[idx]
        eeg = self.eeg_data[idx]
        if self.transform:
            eeg = self.transform(eeg)
        if self.target_transform:
            label = self.target_transform(label)
        return eeg, label

In [ ]:
print(sum(a))
print(len(a))
print(len(a)-sum(a))
print((len(a)-sum(a))/len(a))

1242.0
2880
1638.0
0.56875


### NEEEEEEEEEEEEEEW

In [ ]:
#DE
eeg_DE_dataset = CustomEEGDataset('/content/label_valence_no_neutral_DE.npy','/content/eeg_data_no_neutral_DE.npy' )
trainloader = torch.utils.data.DataLoader(eeg_DE_dataset, batch_size=128,
                                          shuffle=True, num_workers=2)

In [7]:
#PSD
eeg_PSD_dataset = CustomEEGDataset('/content/label_valence_no_neutral_channel_first.npy','/content/eeg_data_no_neutral_PSD_channel_first.npy' )
trainloader = torch.utils.data.DataLoader(eeg_PSD_dataset, batch_size=32,
                                          shuffle=True, num_workers=2)

Batchnormalization CNN

In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.batch_norm1 = nn.BatchNorm2d(5)
        self.batch_norm2 = nn.BatchNorm1d(16)
        #self.batch_norm3 = nn.BatchNorm2d(5)
        #self.batch_norm4 = nn.BatchNorm2d(5)
        self.conv = torch.nn.Conv2d(5, 8, (2,3),padding = 'valid')
        self.conv1 = nn.Conv2d(8, 6, (3,2), padding = 'valid')
        self.conv2 = nn.Conv2d(6, 4, (2,3), padding = 'valid')
        self.conv3 = nn.Conv2d(8, 6, (2,2), padding = 'valid')
        self.pool = nn.MaxPool2d(2, 2)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Conv2d(32, 20, (3,2), padding = 'valid')
        self.conv5 = nn.Conv2d(20, 10, (2,2), padding = 'valid')
        self.fc1 = nn.Linear(120, 16)
        self.fc2 = nn.Linear(16, 1)
        #self.fc3 = nn.Linear(128, 28)
        #self.fc4 = nn.Linear(28, 1)
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.batch_norm1(x)
        x = F.relu(self.conv(x))
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        #x = F.relu(self.conv3(x))
        #x = self.pool2(F.relu(self.conv4(x)))
        #x = F.relu(self.conv5(x))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.batch_norm2(x)
        #x = self.dropout1(x)
        x = self.fc2(x)
        #x = self.batch_norm3(x)
        x = self.dropout2(x)
        #x = F.relu(self.fc3(x))
        #x = self.batch_norm4(x)
        #x = self.fc4(x)
        x = self.sigmoid(x)
        return x

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  correct = 0
  train_acc = []
  for batch_idx, (data, target) in enumerate(train_loader):
      data, target = data.to(device), target.to(device)
      optimizer.zero_grad()
      output = model(data)
      loss = nn.functional.binary_cross_entropy(output, target)
      #loss = nn.BCELoss(output,target)
      loss.backward()
      optimizer.step()
      output = model(data)
      pred = output.round().cpu().detach().numpy()
      target = target.cpu().numpy()
      acc = metrics.accuracy_score(target,pred) #== target).float().mean()
      #print(pred.round())
      #total = int(y.shape[0])
      #batch_accs.append(torch.mean(acc).item())
      train_acc.append(acc)
  return train_acc, loss


net = Net()
net.cuda()


#nn.BatchNorm2d(100)

Net(
  (batch_norm1): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv): Conv2d(5, 8, kernel_size=(2, 3), stride=(1, 1), padding=valid)
  (conv1): Conv2d(8, 6, kernel_size=(3, 2), stride=(1, 1), padding=valid)
  (conv2): Conv2d(6, 4, kernel_size=(2, 3), stride=(1, 1), padding=valid)
  (conv3): Conv2d(8, 6, kernel_size=(2, 2), stride=(1, 1), padding=valid)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(32, 20, kernel_size=(3, 2), stride=(1, 1), padding=valid)
  (conv5): Conv2d(20, 10, kernel_size=(2, 2), stride=(1, 1), padding=valid)
  (fc1): Linear(in_features=120, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=1, bias=True)
  (dropout1): Dropout(p=0.1, inplace=False)
  (dropout2): Dropou

In [36]:
# Define the number of folds and batch size
k_folds = 2
batch_size = 32

# Define the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the k-fold cross validation
kf = KFold(n_splits=k_folds, shuffle=False)

# Loop through each fold
for fold, (train_idx, test_idx) in enumerate(kf.split(eeg_PSD_dataset)):
    print(f"Fold {fold + 1}")
    print("-------")
    all_accs_train = []
    all_loss_train = []
    # Define the data loaders for the current fold
    train_loader = DataLoader(
        dataset=eeg_PSD_dataset,
        batch_size=batch_size,
        sampler=torch.utils.data.SubsetRandomSampler(train_idx),
    )

    test_loader = DataLoader(
        dataset=eeg_PSD_dataset,
        batch_size=batch_size,
        sampler=torch.utils.data.SubsetRandomSampler(test_idx),
    )

    # Initialize the model and optimizer
    model = Net().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001) #betas=(0.5, 0.99))

    # Train the model on the current fold
    for i in range(5):
      for epoch in tqdm(range(1, 200)):
          train_accuracies, loss = train(model, device, train_loader, optimizer, epoch)
          all_accs_train.append(np.mean(train_accuracies))
          all_loss_train.append(loss)

        # Evaluate the model on the test set
      print(all_accs_train[-1])
      print(all_loss_train[-1])
      model.eval()
      test_loss = 0
      correct = 0
      sample_eval = 0
      test_acc_list = []
      with torch.no_grad():
          for data, target in test_loader:
              data, target = data.to(device), target.to(device)
              output = model(data)
              test_loss += nn.functional.binary_cross_entropy(output, target).item()
              pred = output.round().cpu().detach().numpy()
              target = target.cpu().numpy()
              acc = metrics.accuracy_score(target,pred)
              test_acc_list.append(acc)#torch.mean(acc).item()
              sample_eval+= target.shape[0]
      test_loss /= sample_eval
      accuracy = np.mean(test_acc_list)
      print(f"Test set: Average loss: {test_loss:.4f}, Accuracy: ({accuracy:.2f}%)\n")



    test_loss /= sample_eval
    accuracy = np.mean(test_acc_list)
    #Print the results for the current fold
    print(f"Test set: Average loss: {test_loss:.4f}, Accuracy: ({accuracy:.2f}%)\n")

Fold 1
-------


100%|██████████| 199/199 [00:44<00:00,  4.48it/s]


0.5944444444444444
tensor(0.6850, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Test set: Average loss: 0.0217, Accuracy: (0.54%)



100%|██████████| 199/199 [00:45<00:00,  4.41it/s]


0.6291666666666667
tensor(0.7052, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Test set: Average loss: 0.0386, Accuracy: (0.55%)



...

0.65
tensor(0.4861, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Test set: Average loss: 0.0479, Accuracy: (0.47%)



100%|██████████| 199/199 [00:50<00:00,  3.94it/s]


0.6666666666666666
tensor(0.5459, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Test set: Average loss: 0.0480, Accuracy: (0.54%)

Test set: Average loss: 0.0000, Accuracy: (0.54%)

Fold 2
-------


 57%|█████▋    | 113/199 [00:25<00:19,  4.35it/s]


KeyboardInterrupt: 

### Spatial GNN

In [ ]:
!pip install torch-geometric
!pip install tensorboardX

In [55]:
import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader
from torch_geometric.data import Data
import itertools
from tensorboardX import SummaryWriter
from datetime import datetime

Inspiration: https://www.youtube.com/watch?v=-UjytpbqX4A&ab_channel=LindseyAI

In [42]:
class GNNStack(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, task='node'):
        super(GNNStack, self).__init__()
        self.task = task
        self.convs = nn.ModuleList()
        self.convs.append(self.build_conv_model(input_dim, hidden_dim))
        self.lns = nn.ModuleList()
        self.lns.append(nn.LayerNorm(hidden_dim))
        self.lns.append(nn.LayerNorm(hidden_dim))
        for l in range(2):
            self.convs.append(self.build_conv_model(hidden_dim, hidden_dim))

        # post-message-passing
        self.post_mp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim), nn.Dropout(0.25),
            nn.Linear(hidden_dim, output_dim))
        if not (self.task == 'node' or self.task == 'graph'):
            raise RuntimeError('Unknown task.')

        self.dropout = 0.25
        self.num_layers = 3

    def build_conv_model(self, input_dim, hidden_dim):
        # refer to pytorch geometric nn module for different implementation of GNNs.
        if self.task == 'node':
            return pyg_nn.GCNConv(input_dim, hidden_dim)
        else:
            return pyg_nn.GINConv(nn.Sequential(nn.Linear(input_dim, hidden_dim),
                                  nn.ReLU(), nn.Linear(hidden_dim, hidden_dim)))

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        if data.num_node_features == 0:
          x = torch.ones(data.num_nodes, 1)

        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            emb = x
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
            if not i == self.num_layers - 1:
                x = self.lns[i](x)

        if self.task == 'graph':
            x = pyg_nn.global_mean_pool(x, batch)

        x = self.post_mp(x)

        return emb, F.log_softmax(x, dim=1)

    def loss(self, pred, label):
        return F.nll_loss(pred, label)

In [70]:
def train(dataset, task, writer):
    if task == 'graph':
        data_size = len(dataset)
        loader = DataLoader(dataset[:int(data_size * 0.8)], batch_size=64, shuffle=True)
        test_loader = DataLoader(dataset[int(data_size * 0.8):], batch_size=64, shuffle=True)
    else:
        test_loader = loader = DataLoader(dataset, batch_size=64, shuffle=True)

    # build model
    model = GNNStack(max(30, 1), 32, 2, task=task)
    opt = optim.Adam(model.parameters(), lr=0.001)

    # train
    for epoch in range(200):
        total_loss = 0
        model.train()
        for batch in loader:
            #print(batch.train_mask, '----')
            opt.zero_grad()
            embedding, pred = model(batch)
            label = batch.y
            if task == 'node':
                pred = pred[batch.train_mask]
                label = label[batch.train_mask]
            loss = model.loss(pred, label)
            loss.backward()
            opt.step()
            total_loss += loss.item() * batch.num_graphs
        total_loss /= len(loader.dataset)
        writer.add_scalar("loss", total_loss, epoch)

        if epoch % 10 == 0:
            test_acc = test(test_loader, model)
            print("Epoch {}. Loss: {:.4f}. Test accuracy: {:.4f}".format(
                epoch, total_loss, test_acc))
            writer.add_scalar("test accuracy", test_acc, epoch)

    return model


In [71]:
def test(loader, model, is_validation=False):
    model.eval()

    correct = 0
    for data in loader:
        with torch.no_grad():
            emb, pred = model(data)
            pred = pred.argmax(dim=1)
            label = data.y

        if model.task == 'node':
            mask = data.val_mask if is_validation else data.test_mask
            # node classification: only evaluate on nodes in test set
            pred = pred[mask]
            label = data.y[mask]

        correct += pred.eq(label).sum().item()

    if model.task == 'graph':
        total = len(loader.dataset)
    else:
        total = 0
        for data in loader.dataset:
            total += torch.sum(data.test_mask).item()
    return correct / total

In [72]:

def load_graph_data():
    # Load labels and features
    y = np.load('label_valence_no_neutral_channel_first.npy')
    x = np.load('eeg_data_no_neutral_PSD_gamma.npy')
    print(x.shape)
    # Convert to PyTorch tensors
    y = torch.tensor(y, dtype=torch.float32)
    x = torch.tensor(x, dtype=torch.float32)
    y = y.type(torch.LongTensor)

    # fully connected graph for each graph
    edge_index = list(itertools.combinations(range(32), 2))
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

    # Create a list of Data objects
    data_list = [Data(x=x[i], edge_index=edge_index, y=y[i]) for i in range(x.shape[0])]
    print(len(data_list))

    print(data_list[0].x.shape)
    return data_list

In [73]:
data = load_graph_data()

(2880, 32, 30)
2880
torch.Size([32, 30])


In [74]:
writer = SummaryWriter("./log/" + datetime.now().strftime("%Y%m%d-%H%M%S"))

dataset = data
task = 'graph'

model = train(dataset, task, writer)

Epoch 0. Loss: 0.7031. Test accuracy: 0.6198
Epoch 10. Loss: 0.6881. Test accuracy: 0.6198
Epoch 20. Loss: 0.6866. Test accuracy: 0.6198
Epoch 30. Loss: 0.6869. Test accuracy: 0.6198
Epoch 40. Loss: 0.6874. Test accuracy: 0.6198


KeyboardInterrupt: 